# <center>Exploratory Data Analysis</center>

## <center>PAKDD2010_Leaderboard_Data.txt</center>


In [17]:
import pandas as pd
import xlrd
import IPython.display
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from src.visualization import visualize

### Load dataset

In [18]:
#take headers from VariablesList.XLS
headers_df = pd.read_excel('../data/raw/PAKDD2010_VariablesList.XLS') # can also index sheet by name or fetch all sheets
headers = headers_df['Var_Title'].tolist()

print(f'{type(headers)}\n'
      f'{len(headers)} features in original list\n'
      f'{len(set(headers))} unique elements in list')

<class 'list'>
54 features in original list
53 unique elements in list


In [19]:
#correct duplicate headers

headers_unique = []

for header in headers:
    if header not in headers_unique:
        headers_unique.append(header)
    else:
        header_duplicated = header + "_2"
        headers_unique.append(header_duplicated)
        print(f'{header} is duplicated and is replaced by {header_duplicated}')

EDUCATION_LEVEL is duplicated and is replaced by EDUCATION_LEVEL_2


In [20]:
#create train dataframe with headers
X_train = pd.read_csv('../data/raw/PAKDD2010_Leaderboard_Data.txt', sep='\t', encoding= 'unicode_escape', names= headers_unique, low_memory=False, index_col ='ID_CLIENT' )
IPython.display.HTML(X_train.head().to_html())

,CLERK_TYPE,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,QUANT_ADDITIONAL_CARDS,POSTAL_ADDRESS_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,EDUCATION_LEVEL,STATE_OF_BIRTH,CITY_OF_BIRTH,NACIONALITY,RESIDENCIAL_STATE,RESIDENCIAL_CITY,RESIDENCIAL_BOROUGH,FLAG_RESIDENCIAL_PHONE,RESIDENCIAL_PHONE_AREA_CODE,RESIDENCE_TYPE,MONTHS_IN_RESIDENCE,FLAG_MOBILE_PHONE,FLAG_EMAIL,PERSONAL_MONTHLY_INCOME,OTHER_INCOMES,FLAG_VISA,FLAG_MASTERCARD,FLAG_DINERS,FLAG_AMERICAN_EXPRESS,FLAG_OTHER_CARDS,QUANT_BANKING_ACCOUNTS,QUANT_SPECIAL_BANKING_ACCOUNTS,PERSONAL_ASSETS_VALUE,QUANT_CARS,COMPANY,PROFESSIONAL_STATE,PROFESSIONAL_CITY,PROFESSIONAL_BOROUGH,FLAG_PROFESSIONAL_PHONE,PROFESSIONAL_PHONE_AREA_CODE,MONTHS_IN_THE_JOB,PROFESSION_CODE,OCCUPATION_TYPE,MATE_PROFESSION_CODE,EDUCATION_LEVEL_2,FLAG_HOME_ADDRESS_DOCUMENT,FLAG_RG,FLAG_CPF,FLAG_INCOME_PROOF,PRODUCT,FLAG_ACSP_RECORD,AGE,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3,TARGET_LABEL_BAD=1
ID_CLIENT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50001,C,1,Web,0,1,F,1,0,4,SP,Osasco,1,SP,Osasco,Cidade das Flores,Y,5,1.0,3.0,N,0,1537.5,0.0,0,1,0,0,0,0,0,0,0,Y,,NaN,NaN,Y,5,0,9,2.0,NaN,NaN,0,0,0,0,1,Y,48,618,618,NaN
50002,C,20,Web,0,1,F,1,0,4,SP,SAO PAULO,1,SP,Sao Paulo,Jardim Fernandes,Y,5,1.0,4.0,N,0,400.0,0.0,0,0,0,0,0,0,0,0,0,N,,NaN,NaN,N,,0,9,5.0,NaN,NaN,0,0,0,0,1,N,29,358,358,NaN
50003,C,25,Web,0,1,M,1,0,2,PA,BELEM,1,PA,ANANINDEUA,DISTRITO INDUSTRIAL,Y,117,1.0,12.0,N,0,300.0,0.0,0,0,0,0,0,0,0,0,0,N,,NaN,NaN,N,,0,9,4.0,NaN,NaN,0,0,0,0,1,N,53,670,670,NaN
50004,C,10,Web,0,1,F,7,0,3,RN,sao jose de mipibu,1,RN,SAO JOSE DE MIPIBU,BAIRRO NOVO,Y,105,1.0,NaN,Y,0,460.0,0.0,0,0,0,0,0,0,0,0,0,N,,NaN,NaN,N,,0,9,5.0,NaN,NaN,0,0,0,0,1,N,38,591,591,NaN
50005,C,25,Web,0,1,F,1,0,4,SP,SAO PAULO,1,SP,Sao Paulo,Guaianazes,Y,5,1.0,4.0,N,0,400.0,0.0,0,0,0,0,0,0,0,0,0,N,,NaN,NaN,Y,5,0,9,4.0,NaN,NaN,0,0,0,0,2,N,19,844,844,NaN


In [21]:
y_train = pd.read_csv('../data/raw/PAKDD2010_Leaderboard_Submission_Example.txt', sep='\t', encoding= 'unicode_escape', names= ['ID_CLIENT','TARGET_LABEL_BAD=1'], low_memory=False, index_col ='ID_CLIENT')
IPython.display.HTML(y_train.head().to_html())

,TARGET_LABEL_BAD=1
ID_CLIENT,
50001,0.772878
50002,0.177754
50003,0.930523
50004,0.875054
50005,0.117273


In [22]:
print(f'Average probability of bad client: {round(X_train["TARGET_LABEL_BAD=1"].mean(),6)}%')

Average probability of bad client: nan%


In [23]:
X_train['TARGET_LABEL_BAD=1'] = y_train['TARGET_LABEL_BAD=1']

In [24]:
IPython.display.HTML(X_train.head().to_html())

,CLERK_TYPE,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,QUANT_ADDITIONAL_CARDS,POSTAL_ADDRESS_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,EDUCATION_LEVEL,STATE_OF_BIRTH,CITY_OF_BIRTH,NACIONALITY,RESIDENCIAL_STATE,RESIDENCIAL_CITY,RESIDENCIAL_BOROUGH,FLAG_RESIDENCIAL_PHONE,RESIDENCIAL_PHONE_AREA_CODE,RESIDENCE_TYPE,MONTHS_IN_RESIDENCE,FLAG_MOBILE_PHONE,FLAG_EMAIL,PERSONAL_MONTHLY_INCOME,OTHER_INCOMES,FLAG_VISA,FLAG_MASTERCARD,FLAG_DINERS,FLAG_AMERICAN_EXPRESS,FLAG_OTHER_CARDS,QUANT_BANKING_ACCOUNTS,QUANT_SPECIAL_BANKING_ACCOUNTS,PERSONAL_ASSETS_VALUE,QUANT_CARS,COMPANY,PROFESSIONAL_STATE,PROFESSIONAL_CITY,PROFESSIONAL_BOROUGH,FLAG_PROFESSIONAL_PHONE,PROFESSIONAL_PHONE_AREA_CODE,MONTHS_IN_THE_JOB,PROFESSION_CODE,OCCUPATION_TYPE,MATE_PROFESSION_CODE,EDUCATION_LEVEL_2,FLAG_HOME_ADDRESS_DOCUMENT,FLAG_RG,FLAG_CPF,FLAG_INCOME_PROOF,PRODUCT,FLAG_ACSP_RECORD,AGE,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3,TARGET_LABEL_BAD=1
ID_CLIENT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50001,C,1,Web,0,1,F,1,0,4,SP,Osasco,1,SP,Osasco,Cidade das Flores,Y,5,1.0,3.0,N,0,1537.5,0.0,0,1,0,0,0,0,0,0,0,Y,,NaN,NaN,Y,5,0,9,2.0,NaN,NaN,0,0,0,0,1,Y,48,618,618,0.772878
50002,C,20,Web,0,1,F,1,0,4,SP,SAO PAULO,1,SP,Sao Paulo,Jardim Fernandes,Y,5,1.0,4.0,N,0,400.0,0.0,0,0,0,0,0,0,0,0,0,N,,NaN,NaN,N,,0,9,5.0,NaN,NaN,0,0,0,0,1,N,29,358,358,0.177754
50003,C,25,Web,0,1,M,1,0,2,PA,BELEM,1,PA,ANANINDEUA,DISTRITO INDUSTRIAL,Y,117,1.0,12.0,N,0,300.0,0.0,0,0,0,0,0,0,0,0,0,N,,NaN,NaN,N,,0,9,4.0,NaN,NaN,0,0,0,0,1,N,53,670,670,0.930523
50004,C,10,Web,0,1,F,7,0,3,RN,sao jose de mipibu,1,RN,SAO JOSE DE MIPIBU,BAIRRO NOVO,Y,105,1.0,NaN,Y,0,460.0,0.0,0,0,0,0,0,0,0,0,0,N,,NaN,NaN,N,,0,9,5.0,NaN,NaN,0,0,0,0,1,N,38,591,591,0.875054
50005,C,25,Web,0,1,F,1,0,4,SP,SAO PAULO,1,SP,Sao Paulo,Guaianazes,Y,5,1.0,4.0,N,0,400.0,0.0,0,0,0,0,0,0,0,0,0,N,,NaN,NaN,Y,5,0,9,4.0,NaN,NaN,0,0,0,0,2,N,19,844,844,0.117273


In [25]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 50001 to 70000
Data columns (total 53 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   CLERK_TYPE                      20000 non-null  object 
 1   PAYMENT_DAY                     20000 non-null  int64  
 2   APPLICATION_SUBMISSION_TYPE     20000 non-null  object 
 3   QUANT_ADDITIONAL_CARDS          20000 non-null  int64  
 4   POSTAL_ADDRESS_TYPE             20000 non-null  int64  
 5   SEX                             20000 non-null  object 
 6   MARITAL_STATUS                  20000 non-null  int64  
 7   QUANT_DEPENDANTS                20000 non-null  int64  
 8   EDUCATION_LEVEL                 20000 non-null  int64  
 9   STATE_OF_BIRTH                  20000 non-null  object 
 10  CITY_OF_BIRTH                   19990 non-null  object 
 11  NACIONALITY                     20000 non-null  int64  
 12  RESIDENCIAL_STATE           

In [26]:
visualize.missing_values(X_train)

                      NaN Values  % of NaN
EDUCATION_LEVEL_2          19539     97.70
MATE_PROFESSION_CODE       18729     93.64
PROFESSIONAL_CITY          14206     71.03
PROFESSIONAL_BOROUGH       14206     71.03
MONTHS_IN_RESIDENCE         2663     13.32
OCCUPATION_TYPE             2437     12.18
RESIDENCE_TYPE               616      3.08
CITY_OF_BIRTH                 10      0.05

                              Empty Values  % of empty
PROFESSIONAL_PHONE_AREA_CODE         14553       72.76
PROFESSIONAL_STATE                   14287       71.44
RESIDENCIAL_PHONE_AREA_CODE           2268       11.34
PROFESSIONAL_BOROUGH                   290        1.45
PROFESSIONAL_CITY                       63        0.32
STATE_OF_BIRTH                          10        0.05


In [27]:
without_nan = X_train.dropna().shape[0]

print(f'Data without NaN values: {without_nan:,} applicants of a total {X_train.shape[0]:,} corresponding to {round((without_nan)/(X_train.shape[0]),4)*100}%.')

Data without NaN values: 148 applicants of a total 20,000 corresponding to 0.74%.


In [28]:
empty_data = ((X_train.eq(' ')).sum()).sum()
total_rows = X_train.shape[0]*X_train.shape[1]

print(f'Empty Data: {empty_data:,} rows of a total {total_rows:,} corresponding to {round(empty_data/total_rows,2)*100}%')

Empty Data: 31,472 rows of a total 1,060,000 corresponding to 3.0%


In [29]:
print(f'Dataset features datatypes:\n{X_train.dtypes.value_counts()}')

Dataset features datatypes:
int64      25
object     20
float64     8
dtype: int64


In [30]:
cat_features = X_train.select_dtypes('O').nunique()
cat_features_list = list(cat_features.index)
print(f'Categorical columns ({cat_features.shape[0]}):\n{cat_features}')

Categorical columns (20):
CLERK_TYPE                         1
APPLICATION_SUBMISSION_TYPE        2
SEX                                2
STATE_OF_BIRTH                    29
CITY_OF_BIRTH                   5708
RESIDENCIAL_STATE                 27
RESIDENCIAL_CITY                2540
RESIDENCIAL_BOROUGH             8143
FLAG_RESIDENCIAL_PHONE             2
RESIDENCIAL_PHONE_AREA_CODE       78
FLAG_MOBILE_PHONE                  2
COMPANY                            2
PROFESSIONAL_STATE                28
PROFESSIONAL_CITY               1437
PROFESSIONAL_BOROUGH            2540
FLAG_PROFESSIONAL_PHONE            2
PROFESSIONAL_PHONE_AREA_CODE      77
FLAG_ACSP_RECORD                   2
RESIDENCIAL_ZIP_3                754
PROFESSIONAL_ZIP_3               754
dtype: int64


In [31]:
cat_num_features = X_train.select_dtypes("number").nunique()[X_train.select_dtypes("number").nunique()<15]
cat_num_features_list = list(cat_num_features.index)
cat_num_features_list.append('PROFESSION_CODE')
cat_num_features_list.append('MATE_PROFESSION_CODE')
print(f'Categorical numerical columns ({X_train[cat_num_features_list].shape[1]}):\n{X_train[cat_num_features_list].nunique()}')

Categorical numerical columns (26):
PAYMENT_DAY                        6
QUANT_ADDITIONAL_CARDS             3
POSTAL_ADDRESS_TYPE                2
MARITAL_STATUS                     8
EDUCATION_LEVEL                    5
NACIONALITY                        3
RESIDENCE_TYPE                     6
FLAG_EMAIL                         2
FLAG_VISA                          2
FLAG_MASTERCARD                    2
FLAG_DINERS                        2
FLAG_AMERICAN_EXPRESS              2
FLAG_OTHER_CARDS                   2
QUANT_BANKING_ACCOUNTS             3
QUANT_SPECIAL_BANKING_ACCOUNTS     3
QUANT_CARS                         2
OCCUPATION_TYPE                    5
MATE_PROFESSION_CODE              13
EDUCATION_LEVEL_2                  6
FLAG_HOME_ADDRESS_DOCUMENT         2
FLAG_RG                            2
FLAG_CPF                           2
FLAG_INCOME_PROOF                  2
PRODUCT                            3
PROFESSION_CODE                   18
MATE_PROFESSION_CODE              13
dt

In [32]:
number_dtype = X_train.select_dtypes("number").columns
num_features_list = list(set(number_dtype) - set(cat_num_features.index))
num_features_list.remove('PROFESSION_CODE')


num_features = X_train.select_dtypes("number").nunique()[num_features_list]
print(f'Numerical columns ({num_features.shape[0]}):\n{num_features}')

Numerical columns (8):
PERSONAL_MONTHLY_INCOME     4311
MONTHS_IN_THE_JOB             18
MONTHS_IN_RESIDENCE           72
PERSONAL_ASSETS_VALUE        104
AGE                           79
QUANT_DEPENDANTS              16
OTHER_INCOMES                289
TARGET_LABEL_BAD=1         10000
dtype: int64
